![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [15]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import timeit

filename_features = "data/UCI HAR Dataset/features.txt"
filename_labels = "data/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "data/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "data/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "data/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "data/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "data/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "data/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze("columns")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze("columns")
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze("columns")
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [7]:
%%time

clf = DecisionTreeClassifier(random_state=42, ccp_alpha=0.001).fit(X_train, y_train)

acc_train = clf.score(X_train, y_train)
acc_valid = clf.score(X_valid, y_valid)
acc_teste = clf.score(X_test, y_test)

print(f'Acurácia da árvore na base de treino:    {acc_train*100:.1f}')
print(f'Acurácia da árvore na base de validação: {acc_teste*100:.1f}')
print(f'Acurácia da árvore na base de teste:     {acc_teste*100:.1f}')

Acurácia da árvore na base de treino:    98.0
Acurácia da árvore na base de validação: 84.9
Acurácia da árvore na base de teste:     84.9
CPU times: user 3.12 s, sys: 74 µs, total: 3.12 s
Wall time: 3.12 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [14]:
%%time

n = 1

prcomp = PCA(n_components=n).fit(X_train)

colunas = ['cp'+str(x+1) for x in list(range(n))]
pc_treino = pd.DataFrame(prcomp.transform(X_train), columns = colunas)
pc_valida = pd.DataFrame(prcomp.transform(X_valid), columns = colunas)
pc_teste  = pd.DataFrame(prcomp.transform(X_test), columns = colunas)


clf = DecisionTreeClassifier(random_state=42, ccp_alpha=0.001).fit(pc_treino, y_train)

acc_train = clf.score(pc_treino, y_train)
acc_valid = clf.score(pc_valida, y_valid)
acc_teste = clf.score(pc_teste, y_test)

print(f'Acurácia da árvore na base de treino:    {acc_train*100:.1f}')
print(f'Acurácia da árvore na base de validação: {acc_teste*100:.1f}')
print(f'Acurácia da árvore na base de teste:     {acc_teste*100:.1f}')

Acurácia da árvore na base de treino:    50.1
Acurácia da árvore na base de validação: 46.1
Acurácia da árvore na base de teste:     46.1
CPU times: user 1.25 s, sys: 7.5 ms, total: 1.26 s
Wall time: 325 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [20]:
prcomp = PCA().fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_valida = prcomp.transform(X_valid)
pc_teste  = prcomp.transform(X_test)

ns = [1, 2, 5, 10, 50]

for n in ns:
    start = timeit.default_timer()
    colunas = ['cp'+str(x+1) for x in list(range(n))]
    df_treino = pd.DataFrame(pc_treino[:,:n], columns = colunas)
    df_valida = pd.DataFrame(pc_valida[:,:n], columns = colunas)
    df_teste  = pd.DataFrame(pc_teste[:,:n], columns = colunas)

    clf = DecisionTreeClassifier(random_state=42, ccp_alpha=0.001).fit(df_treino, y_train)
    stop = timeit.default_timer()

    acc_train = clf.score(df_treino, y_train)
    acc_valid = clf.score(df_valida, y_valid)
    acc_teste = clf.score(df_teste, y_test)


    print(f"N PCA: {n} | ", end="")
    print(f"Tempo: {stop - start:.2f}s")
    print(f'Acurácia da árvore na base de treino:    {acc_train*100:.1f}')
    print(f'Acurácia da árvore na base de validação: {acc_teste*100:.1f}')
    print(f'Acurácia da árvore na base de teste:     {acc_teste*100:.1f}')
    print("\n======\n")



N PCA: 1 | Tempo: 0.04s
Acurácia da árvore na base de treino:    50.0
Acurácia da árvore na base de validação: 45.7
Acurácia da árvore na base de teste:     45.7


N PCA: 2 | Tempo: 0.03s
Acurácia da árvore na base de treino:    61.3
Acurácia da árvore na base de validação: 58.5
Acurácia da árvore na base de teste:     58.5


N PCA: 5 | Tempo: 0.04s
Acurácia da árvore na base de treino:    84.6
Acurácia da árvore na base de validação: 78.9
Acurácia da árvore na base de teste:     78.9


N PCA: 10 | Tempo: 0.09s
Acurácia da árvore na base de treino:    89.3
Acurácia da árvore na base de validação: 82.4
Acurácia da árvore na base de teste:     82.4


N PCA: 50 | Tempo: 0.47s
Acurácia da árvore na base de treino:    91.9
Acurácia da árvore na base de validação: 82.3
Acurácia da árvore na base de teste:     82.3




## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

A acurácia vai subindo da mesma forma que o tempo